In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [3]:
url = "https://classiccars.com/"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

response = requests.get(url, headers=headers)

In [4]:
bsobj = BeautifulSoup(response.content, "html.parser")
#bsobj

In [19]:
list_title = bsobj.find_all("div", {"class": "h-title"})
list_title_clean = [s.get_text().strip for s in list_title if s.get_text()[:4].isdigit()]
list_title_clean

['1970 Plymouth Barracuda',
 '1969 Dodge Charger',
 '1967 Chevrolet Camaro',
 '1966 Chevrolet Chevelle',
 '1962 Chevrolet Corvette',
 '1974 Ford Bronco',
 '1966 Chevrolet Corvette Stingray',
 '1958 Cadillac Series 62',
 '1971 Pontiac Firebird',
 '1969 Dodge Charger',
 '1965 Ford F100',
 '1973 Plymouth Satellite',
 '1970 Dodge Challenger',
 '1957 Chevrolet Bel Air Convertible',
 '1937 Cord Phaeton',
 '1969 Ford Bronco',
 '1976 Chevrolet Blazer',
 '1970 Chevrolet K-20',
 '1962 Chevrolet Impala',
 '1978 MG MGB']

In [21]:
list_price = bsobj.find_all("div", {"class": "p-price"})
list_price_string = [price.get_text().split()[0].strip("$").replace(",","") for price in list_price]
list_price_float = []

list_price_string

['38000',
 '79900',
 '115000',
 '72500',
 '225000',
 '110000',
 '174999',
 '108000',
 '41000',
 '97750',
 '22488',
 '24995',
 '127995',
 '109995',
 '117500',
 '104995',
 'View',
 'View',
 'View',
 'View',
 'View',
 'View',
 'View',
 'View',
 'View']

In [7]:
# 

In [23]:
cards = bsobj.find_all("div", {"class":"h-card"})
cards

[<div class="fx-item fx-justify h-card w100 pos-rel top-pad" id="PC-Nav" style="min-height: 24px;">
 <a aria-label="Find a Car" class="u-url navbar-link" href="/search/advanced" id="navFind01">Search</a>
 <a aria-label="Sell a Car" class="u-url navbar-link" href="/sell-my-classic-car" id="navSell01">Sell a Car</a>
 <a aria-label="Find Dealers" class="u-url navbar-link" href="/dealer-directory/map" id="navMap">Find Dealers</a>
 <a aria-label="Join Our Dealers" class="u-url navbar-link" href="/join-the-dealer-network" id="navDealerPitch">Join our Dealers</a>
 <a aria-label="Auction Central" class="u-url navbar-link" href="/auction" id="navAuctions01">Auction Central</a>
 <a aria-label="Resource Guide" class="u-url navbar-link" href="/resource-guide" id="navResources01">Resources</a>
 <a aria-label="The Journal" class="u-url navbar-link" href="https://journal.classiccars.com/" id="navJournal01" rel="noopener" target="_blank">Journal</a>
 <a aria-label="AutoHunter Live Auctions" class="u-u

In [45]:
def fetch_image(url, title, folder_name):
    extension = os.path.splitext(url)[-1]
    file_name = f"{title}{extension}"
    file_path = os.path.join(folder_name, file_name)

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    with open(file_path, "wb") as image_file:
        image_file.write(response.content)
    return True, file_name 

In [47]:
fetch_image(url, "title2", "car_images")

(True, 'title2')

In [53]:
card_list = []

for card in cards:
    try:
        title = card.find("div", {"class":"h-title"}).get_text().strip()
        img_url = card.find("img")['data-src']
        fetch_image(img_url, title.replace(" ", "-"), "car_images")
    except:
        title = "No title"
    try:
        price_raw = card.find("div", {"class":"p-price"}).get_text()
        price = float(price_raw.split()[0].strip("$").replace(",",""))
    except:
        price = 0
    if title != "No title" and price > 0:
        card_list.append((title, price))

card_list

[('1970 Plymouth Barracuda', 38000.0),
 ('1969 Dodge Charger', 79900.0),
 ('1967 Chevrolet Camaro', 115000.0),
 ('1966 Chevrolet Chevelle', 72500.0),
 ('1962 Chevrolet Corvette', 225000.0),
 ('1974 Ford Bronco', 110000.0),
 ('1966 Chevrolet Corvette Stingray', 174999.0),
 ('1958 Cadillac Series 62', 108000.0),
 ('1971 Pontiac Firebird', 41000.0),
 ('1969 Dodge Charger', 97750.0),
 ('1965 Ford F100', 22488.0),
 ('1973 Plymouth Satellite', 24995.0),
 ('1970 Dodge Challenger', 127995.0),
 ('1957 Chevrolet Bel Air Convertible', 109995.0),
 ('1937 Cord Phaeton', 117500.0),
 ('1969 Ford Bronco', 104995.0)]

In [55]:
df = pd.DataFrame(card_list, columns=["title", "price"])
df.head()

,title,price
0,1970 Plymouth Barracuda,38000.0
1,1969 Dodge Charger,79900.0
2,1967 Chevrolet Camaro,115000.0
3,1966 Chevrolet Chevelle,72500.0
4,1962 Chevrolet Corvette,225000.0


In [57]:
# use apply and lambda to split year into a separate column

In [59]:
df["year"] = df["title"].apply(lambda s: s[:4])
df["title"] = df["title"].apply(lambda s: s[5:])
df = df.reindex(columns=["year","title","price"])

In [61]:
df.iloc[0].to_dict()

{'year': '1970', 'title': 'Plymouth Barracuda', 'price': 38000.0}

In [63]:
df.head()

,year,title,price
0,1970,Plymouth Barracuda,38000.0
1,1969,Dodge Charger,79900.0
2,1967,Chevrolet Camaro,115000.0
3,1966,Chevrolet Chevelle,72500.0
4,1962,Chevrolet Corvette,225000.0
